In [10]:
from flask import Flask,render_template,url_for,request
import pandas as pd 
import pickle
import requests
from bs4 import BeautifulSoup



In [ ]:

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/predict',methods=['POST'])
def predict():
    if request.method == 'POST':
        headline = request.form['message']
        print(type(headline))
        data = [headline]
        data = [l.split(',') for l in ','.join(data).split('\r\n')]
        data=[y for x in data for y in x]
        df=model_predict(data,1)
    return render_template('censor.html',prediction = df)

@app.route('/scrape',methods=['POST'])
def scrape():
    if request.method == 'POST':
        url = request.form['message']
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        #scraping headlines
        headlines = soup.find_all(attrs={"itemprop": "headline"})
        dataset=[]
        for headline in headlines:
            dataset.append(headline.text)
        #print(dataset)
        #scraping links
        links=[]
        link_headlines = soup.find_all("span")
        
        for link_headline in link_headlines:
            var=link_headline.get('itemid')
            
            if var==None:
                continue
            else:
                links.append(var)
        
        df=model_predict(dataset,0,links)
        #print(df)
        df_dict=dict(zip(df.headline, df.links))
        #print(df_dict)
        #pred = {dataset[i]: test_values[i] for i in range(len(test_keys))} 
    return render_template('scrape_censor.html',prediction = df_dict,link=links)

def model_predict(data,flag,links=None):
    tr=open('headline_transform.pkl','rb')
    tr=pickle.load(tr)
    t_model= open('censor_predict.pkl','rb')
    t_model=pickle.load(t_model)

    test=pd.Series(data)
    
    test=tr.transform(test)
    pred=t_model.predict(test)
    if flag==0:
        print("scrappping")
        df=pd.DataFrame({'headline':data,'prediction':pred,'links':links})
        df=df[['headline','links']].where(df['prediction']==0).dropna()
    if flag==1:
        print("only predict")
        df=pd.DataFrame({'headline':data,'prediction':pred})
        df=df['headline'].where(df['prediction']==0).dropna()
    return df
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Oct/2019 11:18:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2019 11:18:16] "POST /scrape HTTP/1.1" 200 -


scrappping


In [1]:
ls=['abc\r\ndef\r\nghi']
ls

['abc\r\ndef\r\nghi']

In [2]:
ls.split()

AttributeError: 'list' object has no attribute 'split'

In [3]:
import itertools

delim = '\r\n'
smallerlist = [list(y) for x, y in itertools.groupby(ls, lambda z: z == delim)
                if not x]

In [4]:
smallerlist

[['abc\r\ndef\r\nghi']]

In [5]:
smallerlist = [l.split(',') for l in ','.join(ls).split('\r\n')]
sl=[y for x in smallerlist for y in x]
sl

['abc', 'def', 'ghi']

In [6]:
pred=[0,1,0]

In [8]:
links=['a','b','c']

In [30]:
df=pd.DataFrame({'headline':sl,'prediction':pred})
df=df.where(df['prediction']==0)
print(df)
dict(zip(df.headline, df.prediction))

  headline  prediction
0      abc         0.0
1      NaN         NaN
2      ghi         0.0


{'abc': 0.0, nan: nan, 'ghi': 0.0}

In [24]:
test=pd.Series(smallerlist)

In [25]:
test

0    [abc]
1    [def]
2    [ghi]
dtype: object

In [35]:
flatten = [y for x in test for y in x]
flatten

['abc', 'def', 'ghi']

In [42]:
testa=pd.Series(flatten)
testa.shape[0]

3

In [29]:
test=tr.transform(test)
pred=t_model.predict(test)
pred[0]

NameError: name 'tr' is not defined